## Pdf Upload

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Resat-Nuri-Guntekin-Calikusu.pdf")
data = loader.load()

In [3]:
len(data)

298

## Chunking

* In this way, we divide the document into smaller sections and access the correct information more easily.
* In many RAG-based projects, the chunk size is chosen between 512 and 2000.

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)

print("Total", len(docs))

Total 935


In [6]:
# To see some values in the document

docs[8]

Document(metadata={'producer': 'calibre 1.22.0 [http://calibre-ebook.com]', 'creator': 'calibre 1.22.0 [http://calibre-ebook.com]', 'creationdate': '2018-06-07T15:24:58+00:00', 'author': 'Reşat Nuri Güntekin', 'keywords': 'Roman, Türk Klasik, Cumhuriyet Dönemi', 'title': 'Çalıkuşu', 'source': 'Resat-Nuri-Guntekin-Calikusu.pdf', 'total_pages': 298, 'page': 7, 'page_label': '8'}, page_content='sedasız, ağırbaşlı ve upuzun bir komşu.\nÖte yanımda manastır terbiyesinin istediği serin ve mağrur loşluğu temin için yapılmışa benzeyen\nve panjurları hiç açılmayan bir uzun pencere dururdu. Ehemmiyetli bir keşif yapmıştım. Göğsümü\nsıraya yaslayıp çenemi biraz yukarı kaldırdığım vakit panjurların arasından gökyüzünün bir\nparçasıyla bir büyük akasyanın yaprakları arasından tek bir apartman penceresi ve bir balkon\nparmaklığı görünürdü\nDoğrusunu söylemek lâzım gelirse, manzara hiç de zengin değildi. Pencere her zaman kapalı durur,\nbalkon parmaklığına hemen daima bir ufak çocuk şiltesi ile yorga

## Creating Embedding using Google Generative AI

In [8]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv  

load_dotenv()

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
# Test (Convert text to vector)
vector = embeddings.embed_query("hello, world!")
vector[:5]

[0.014134909026324749,
 -0.022324152290821075,
 -0.054603420197963715,
 -0.006284549366682768,
 -0.03392402455210686]

## Recording on ChromaDB

In [10]:
vectorstore = Chroma.from_documents(documents=docs, embedding = embeddings)

In [11]:
# The Similarity feature takes embedding vectors and finds the closest vectors in Chroma.
# For example: When we say "What is encoder?", it brings up the documents that contain the concept of "encoder" or are closest to it.
# The value in the search results tells us how many values will be returned.
# For imstance: If k=10, it brings us the 10 most relevant values.

retriever = vectorstore.as_retriever(
    search_type="similarity", search_kwargs={"k": 10}
)

In [12]:
retrieved_docs = retriever.invoke("Feride kimdir?")

In [13]:
len("retrieved_docs")

14

In [14]:
print(retrieved_docs[5].page_content)

-Teyze çocukları hemen hemen kardeş demektir. Feride’nin erkek kardeşi olmadığı için sen,
doğrudan doğruya onun ağabeyi sayılırsın Kâmran; kardeşine “Hoş geldin,” desene!...
Kâmran hâlâ bir şey söyleyemiyordu. Hafifçe eğildi, Feride’nin saçlarına dudaklarını dokundurdu.
Sonra kulağına söyler gibi gayet yavaş:
-Sizi tekrar görmek memnuniyetini söyleyebilmek için kelime bulamayacağım Feride Hanım, dedi.


### LLM invoke operations using Google Gemini API structure

 *  **Max_tokens:** Max_tokens is the maximum length of the response produced by the model.

 *  **Temperature:** Temperature determines the randomness and creativity in the model's responses.

**Temperature Value Range:** 

* **Low Values(0.1-0.3)**: It allows us to give more precise and consistent answers.

* **Medium Values(0.4-0.7)**: It gives both logical and creative answers.

* **High Values(0.8-1.0)**: It may give more random and creative answers and sometimes perhaps inconsistent answers.



In [16]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.3,
    max_tokens=500
)

In [17]:
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain


In [18]:
system_prompt = ( "You are an assistant for question-answering tasks"
                  "Use the following pieces of retrieved context to answer"
                  "If you don't know the answer, say that you don't know"
                  "Use three sentences maximum and keep the answer corrects."
                  "\n\n"
                  "{context}"
)

In [19]:
prompt = ChatPromptTemplate.from_messages(
    [
       ("system", system_prompt),
       ("human",  "{input}")
        
    ]
    
)

## Creating a Question-Answer Chain (LLM + Prompt)

In [21]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)

## Creating a RAG Chain (RAG + LLM Merge)

In [23]:
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

## Generating Answers by Running a User Query

In [25]:
response = rag_chain.invoke({"input": "Feride kimdir? Karakter özellikleri nedir?"})

In [26]:
print(response["answer"])

Feride, Çalıkuşu romanının ana karakteridir.  Enerjik, şakacı, eğlenceli, merhametli ve biraz da çocuksu bir kişiliğe sahiptir.  Aynı zamanda fevri,  duygusal ve  idealisttir.
